# From unstructered data to it Provenance


<div style="text-align: right"> 
<h3>EScience - PPG IC/UFF 2017.1</h3>

<h4>Daniel Junior, Kid Valeriano</h4>

</div>


## Relembrando...
O projeto tem como base o artigo <strong>Provenance as data mining: Combining file system metadata with content analysis</strong>.
</hr>
<li>A ideia é utilizar técnicas de Mineração de Dados para reconstruir a proveniência de dados não-estruturados.</li>
<li>Os dados não-estruturados utilizados como dados de entrada para o projeto são arquivos textos contendo versões de um artigo até que o mesmo chegasse a sua edição final.</li>

## Arquitetura
<div width="100%" height="100%">
    <img src="images/arquitetura.png" height="400" width="800" style="margin: 0 auto;"/>
</div>

## Projeto
<ul>
    <li>Linguagem Python</li>
    <li>Biblioteca Sci-kit Learn</li>
    <li>Os dados usados no experimento são do repositório <strong><a href="https://l.facebook.com/l.php?u=http%3A%2F%2F137.207.234.78%2Fsearch%2Fdetail%2F&h=ATMRd4R-lyI7h1CWdhxvn6UWFuIQ6Sk5lh5mt_Yl5P4cZtgMLg2Eab3JBX_fKeG0-K69PJBwnMqh8BrtHvD__C71t0gmJg-NTkGuLNtxIehOx2CwfwQx_44sEEALKLVSY6y_fg">CiteSeerX</a></strong></li>
</ul>

## Projeto

Os passos a serem seguidos são:

<ol>
    <li>Dado um diretório que contenha as revisões de arquivos, devo ser capaz de recuperar o conteúdo de cada arquivo.</li>
    <li>Representar o conteúdo do arquivo de uma forma que seja possível aplicar algoritmos de Clustering.</li>
    <li>Executar o algoritmo para criar os clusters.</li>
    <li>Para cada arquivo em um arquivo, recuperar os metados relativos ao dados de modificação do arquivo.</li>
    <li>Com os metadados em mãos, exportar para o PROV-Model, fazendo o armazenamento em um banco relacional (SQLite).</li>
    <li>Ser capaz de receber um arquivo de entrada e retornar uma representação gŕafica da proveniência do mesmo.</li>
</ol>


In [8]:
#imports
import os, time
from stat import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import train_test_split

### Passo 1: Carregar os dados

In [4]:
PATH = "data"
filenames = []
docs = []
for path, dirs, files in os.walk(PATH):
    for filename in files:
        fullpath = os.path.join(path, filename)
        with open(fullpath, 'r') as f:
            data = f.read()
            filenames.append(fullpath)
            docs.append(data)        

print(filenames[:10])

['data/d2/doc684.txt', 'data/d2/doc641.txt', 'data/d2/doc43.txt', 'data/d2/doc752.txt', 'data/d2/doc167.txt', 'data/d2/doc786.txt', 'data/d2/doc154.txt', 'data/d2/doc293.txt', 'data/d2/doc272.txt', 'data/d2/doc1142.txt']


In [11]:
train_data, test_data = train_test_split(docs, test_size=0.2, random_state=42)

### Passo 2: Representação vetorial dos dados

<div width="100%" height="100%">
    <img src="images/tfidf.png" style="margin: 0 auto;"/>
</div>

### Passo 2: Representação vetorial dos dados

In [20]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=200000,)
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data)
print("Dimensões da matriz:")
print(tfidf_matrix.todense().shape)

Dimensões da matriz:
(928, 20202)


Before all we need to load data, as follow:

In [4]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans

In [5]:
categories = [
    'talk.politics.mideast',
    'rec.sport.baseball',
]

train_data = fetch_20newsgroups(subset='train', categories=categories)
test_data = fetch_20newsgroups(subset='test', categories=categories)
labels = train_data.target_names
print(labels)

['rec.sport.baseball', 'talk.politics.mideast']


In [22]:


PATH = "data"
filenames = []
docs = []
#getting all path and files contents from PATH variable
for path, dirs, files in os.walk(PATH):
    for filename in files:
        fullpath = os.path.join(path, filename)
        with open(fullpath, 'r') as f:
            data = f.read()
            #appending filepath and content to array
            filenames.append(fullpath)
            docs.append(data)        

#getting metadata from file
st = os.stat(filenames[0])
print("file size:", st[ST_SIZE])
print("file modified:", time.asctime(time.localtime(st[ST_MTIME])))

file size: 3580
file modified: Thu May 18 11:02:46 2017


After that, we will use TF-IDF to documents representation.

In [6]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data.data)
terms = tfidf_vectorizer.get_feature_names()
print(terms)

NameError: name 'train_data' is not defined

Then lets instance K-Means model and fit with TF-IDF matrix that we retrieve from last step

In [7]:
km = KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1,
                verbose=False)
km.fit(tfidf_matrix.todense())
print(km.labels_)

[0 0 1 ..., 0 1 0]


Now we are able to inspect clusters centers (showing only first 10 attributes) and make predictions on test data

In [8]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(2):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()
    
km.predict(tfidf_vectorizer.transform(test_data.data).todense())

Cluster 0: edu year com baseball team game organization subject lines article
Cluster 1: israel israeli edu jews turkish people armenian armenians com arab


array([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0,

In [14]:

#for idx, doc in enumerate(train_data.data):
 #   f = open('data/doc'+str(idx)+".txt", 'w')
  #  f.write(doc)
   # f.close()

In [3]:
from IPython.core.display import HTML
from urllib.request import urlopen
HTML(urlopen('https://raw.githubusercontent.com/lmarti/jupyter_custom/master/custom.include').read().decode('utf-8'))